# @decorators in Python
## What are decorators?
Python provides a feature of decorators which basically **adds functionality to an existing code**. It takes in a function, adds some functionality to it and then returns an augmented function with those added functionalities.<br>
This allows the user to perform **metaprogramming** i.e. the concept when a part of the program tries to modify another part of the program at compile time.

In [1]:
def my_decorator_function(function_to_decorate):
    def decorated_function():
        # Adding stars as "decoration"
        print('*'*25)
        function_to_decorate()
        print('*'*25)
    return decorated_function

@my_decorator_function
def function_to_decorate():
    print("I am an ordinary message.")

def function_without_decoration():
    print("I am an ordinary message.")

In [2]:
function_to_decorate()

*************************
I am an ordinary message.
*************************


In [3]:
function_without_decoration()

I am an ordinary message.


The `@` symbol along with the name of the decorator function is how decorators are used.<br>

## How and Why do decorators work?
In Python, EVERYTHING is an object including the functions or the classes. The variables we use are identifers bound to these objects.<br>
Functions, being objects, can be assigned to other variables (as objects) as shown below:

In [4]:
def my_func():
    print("my function's message")

my_func()

my function's message


In [5]:
xyz = my_func

xyz()

my function's message


Now, in Python functions can also be passed as arguments! This was how we were able to "decorate" it in the first code snippet.

In [6]:
def times5(num):
    return num*5


def times10(num):
    return num*10


def multiplier(function, number):
    result = function(number)
    return result

In [7]:
print(multiplier(times5,10))
print(multiplier(times10,10))

50
100


Lastly, in Python we can define and return a function inside a function.

In [8]:
def function_called():
    def function_returned():
        print("Inner Function.")
    return function_returned

my_func = function_called()

my_func()

Inner Function.


These are the basic prerequisites for understanding Decorators in Python. To summarize:
1. **functions** (infact everything) **in Python are objects**.
2. a **function can be passed as an argument in another function**.
3. a **function can be defined and returned by a function** (known as a **high order function**).

## Back to Decorators...

Functions and methods are called callable since they can be called.<br>
Any object which implements the special `__call__()` method in Python is termed callable. In the most basic sense, **a decorator is a callable that returns a callable**.

In [9]:
def decorate(function):
    def inner_func():
        print("Decorated!")
        function()
    return inner_func


def simple_msg():
    print("Simple Message.")

In [10]:
simple_msg()

Simple Message.


In [11]:
result = decorate(simple_msg)
result()

Decorated!
Simple Message.


Now the above code is simply passing a function as argument and performing the task. Using decorators it will look like this:

In [12]:
@decorate
def my_msg():
    print("My Message.")

my_msg()

Decorated!
My Message.


## Decorating Functions with Parameters
Suppose a function defined as:
```
def find_max(x,y):
    if(x>y):
        return x
    else
        return y
```
needs to be decorated using our `@decorate` decorator. It would result in a `TypeError` since there is mismatch in the no. of arguments within our decorator and the original function.

In [13]:
@decorate
def find_max(x,y):
    if(x>y):
        print(x)
    else:
        print(y)

find_max(10,20)

TypeError: inner_func() takes 0 positional arguments but 2 were given

Now the way to go about it is to either create a decorator with the same number of arguments as the function. However this also has its drawbacks like if we have a new function with different number of arguments for the same decorator.. we are again stuck with the same problem as before.

In [ ]:
def decorate(function):
    def inner_func(x,y):
        print("Decorated!")
        function(x,y)
    return inner_func

In [ ]:
@decorate
def find_max(x,y):
    if(x>y):
        print(x)
    else:
        print(y)

find_max(10,20)

The Drawback:

In [ ]:
@decorate
def find_avg(x,y,z):
    print((x+y+z)/3)

find_avg(10,20,30)

### using `*args` and `**kwargs`
We can create general decorators that work with any number of parameters.<br>
In Python, this is done as `function(*args, **kwargs)` where, args will be the tuple of positional arguments and kwargs will be the dictionary of keyword arguments. It can be implemented as:

In [ ]:
def decorate(function):
    def inner_func(*args,**kwargs):
        print("Decorated!")
        print(args)
        print(kwargs)
        function(*args, **kwargs)
    return inner_func

In [ ]:
@decorate
def find_max(x,y):
    if(x>y):
        print(x)
    else:
        print(y)

find_max(10,20)

In [ ]:
@decorate
def find_avg(x,y,z):
    print((x+y+z)/3)

find_avg(10,20,30)

## Chaining Decorators in Python
Multiple decorators can be chained in Python i.e. a function can be decorated multiple times with different or same decorators by simply placing the decorators above the desired function.

In [ ]:
def star_decoration(func):
    def inner(*args, **kwargs):
        print("*"*15)
        func(*args, **kwargs)
        print("*"*15)
    return inner


def hash_decoration(func):
    def inner(*args, **kwargs):
        print("#"*15)
        func(*args, **kwargs)
        print("#"*15)
    return inner

In [ ]:
@star_decoration
@hash_decoration
def my_msg_printer(msg):
    print(msg)

my_msg_printer("I am a message!")

The above snippet of,
```
@star_decoration
@hash_decoration
def my_msg_printer(msg):
    print(msg)
```
is equivalent to
```
def my_msg_printer(msg):
    print(msg)
my_msg_printer = star_decoration(hash_decoration(my_msg_printer))
```
It must be noted that the order in which we chain decorators matter.
```
@star_decoration
@hash_decoration
def my_msg_printer(msg):
    print(msg)
```
is different from
```
@hash_decoration
@star_decoration
def my_msg_printer(msg):
    print(msg)
```

## Decorator with arguments (decorator factory)
We know that a decorator takes just one argument i.e. the function to be decorated and there is no way to pass other arguments.
However it can be made possible by creating a function which takes arbitrary arguments and returns a decorator.

In [ ]:
def decoratorfactory(message):
    def decorator(func):
        def inner(*args, **kwargs):
            print('Message passed to the decorator: {}'.format(message))
            return func(*args, **kwargs)
        return inner
    return decorator

In [ ]:
@decoratorfactory('Decorator Factory')
def my_func():
    print("my function's message")

my_func()

## Decorator class
As mentioned before, a decorator is a function that can be applied to another function to augment its
behavior.<br>
But what if the decorator was instead a class? The syntax would still work, except that now function will get replaced with an instance of the decorator class.<br><br>
Now, if the decorator class implements the `__call__()` method, then it would still be possible to use the function as it was.

In [ ]:
class Decorator(object):
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print('my decorator')
        return self.func(*args, **kwargs)

In [ ]:
@Decorator
def my_func():
    print('my function')

my_func()

**Note**: A function decorated with a class decorator will no longer be considered a "function". If we test it's Type it will no longer be a `FunctionType`.

### Decorating Class Methods
For decorating class methods an additional `__get__` method is defined.

In [ ]:
from types import MethodType

class Decorator(object):
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print('my decorator')
        return self.func(*args, **kwargs)
    def __get__(self, instance, cls):
        return self if instance is None else MethodType(self, instance)

In [ ]:
class Test(object):
    def __init__(self):
        pass
    @Decorator
    def my_test_method(self):
        print("my test method")

In [ ]:
test = Test()
test.my_test_method()